In [1]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
from spacy.tokens import DocBin
import json
from youtube_transcript_api import YouTubeTranscriptApi
import string
import contractions
from sklearn.metrics import confusion_matrix, classification_report
from googleapiclient.discovery import build
import time
import os
import sys


In [4]:
def create_train_test_data(files,file_name):
    classe = []
    annotations = []
    count = 0
    for file in files:
        root = os.getcwd().split('wanderly.ai')[0]
        file_path = os.path.join(root, f'wanderly.ai/data/annotations/')
        if not os.path.exists(file_path):
            os.mkdir(file_path)
        json_f = open(file_path + file)

        nlp = spacy.blank("en")
        training_data = json.load(json_f)
        for i in training_data['classes']:
            classe.append(i)
        for i in training_data['annotations']:
            annotations.append(i)
        count = count + len(training_data['annotations'])
    final_json_file = {'classe':list(set(classe)),'annotations':annotations}
    root = os.getcwd().split('wanderly.ai')[0]
    file_path = os.path.join(root, f'wanderly.ai/data/')
    if not os.path.exists(file_path):
        os.mkdir(file_path)
    with open(file_path + f'{file_name}.json', "w") as file:
        json.dump(final_json_file, file, indent=2)

    json_f = open(file_path + f'{file_name}.json')

    training_data = json.load(json_f)
    
    # the DocBin will store the example documents
    db = DocBin()
    compleated=False
    missing_sent = []
    i=-1
    while not compleated:
        i = i+1
        run = 0
        try:
            for text, annotations in tqdm(training_data['annotations'][i:]):
                try:
                    run = run+1
                    doc = nlp.make_doc(text)
                    ents = []
                    for start, end, label in annotations['entities']:
                        span = doc.char_span(start, end, label=label, alignment_mode='contract')
                        ents.append(span)
                    doc.ents = ents
                    db.add(doc)
                except:
                    print(text)
            
            db.to_disk(file_path + f"{file_name}.spacy")
            print("Saved Success")
            compleated = True
        except:
            print('error')
            i = i + run 



In [5]:
annotations_files_train = [
    'annotations_7cPLbiblb84.json',
    'annotations_p0MvovsCxCk.json',
    'annotations.json',
    'annotations_5zA6OFpkPe0.json',
]
annotations_files_test = [
    
    'annotations_Lv0PkSkKeSo.json',
    'annotation_rm_j6O8y148.json'
]
create_train_test_data(annotations_files_train,'train_data')
create_train_test_data(annotations_files_test,'test_data')

 66%|██████▌   | 269/410 [00:00<00:00, 1162.36it/s]


error


  2%|▏         | 3/140 [00:00<00:00, 750.64it/s]


error


  0%|          | 0/136 [00:00<?, ?it/s]


...

error


100%|██████████| 93/93 [00:00<00:00, 1401.76it/s]


Saved Success


100%|██████████| 185/185 [00:00<00:00, 1187.77it/s]

Saved Success


In [6]:
! python -m spacy init fill-config ../configs/base_config.cfg ../configs/config.cfg

✔ Auto-filled config with all values
✔ Saved config
..\configs\config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [71]:
# Model Training
! python -m spacy train ../configs/config.cfg --output ../ --paths.train ../data/train_data.spacy --paths.dev ../data/test_data.spacy

^C


### Model Testing 

In [ ]:
json_f = open(F'D:/randomProjects/wamderly/data/annotations/annotations_7cPLbiblb84.json')
training_data = json.load(json_f)
nlp = spacy.blank("en")
i = -1
basic_database = dict()
compleated=False
while not compleated:
        i = i+1
        run = 0
        try:
            for text, annotations in tqdm(training_data['annotations'][i:]):
                try:
                    run = run+1
                    for start, end, label in annotations['entities']:
                        if label in basic_database.keys(): 
                            basic_database[str(label)].append(str(text[start:end]))
                        else:
                            basic_database[label] = []
                            basic_database[str(label)].append(str(text[start:end]))
                except:
                    # print(text)
                    continue
            # db.to_disk(f"D:/randomProjects/wamderly/data/{file_name}.spacy")
            print("Saved Duccess")
            compleated = True
        except:
            print('error')
            i = i + run 
basic_database


# Loading trained model
nlp = spacy.load('../models/ner_mod/model-best')

json_f = open(F'D:/randomProjects/wamderly/data/annotations/annotations_7cPLbiblb84.json')
training_data = json.load(json_f)

test_data = training_data['annotations']

y_true = []
y_pred = []

for text, annotations in test_data:
    doc = nlp(text)
    pred_entities = {(ent.start_char, ent.end_char): ent.label_ for ent in doc.ents}
    true_entities = {(start, end): label for start, end, label in annotations["entities"]}

    all_offsets = set(pred_entities.keys()) | set(true_entities.keys())

    for offset in all_offsets:
        y_true.append(true_entities.get(offset, "O"))
        y_pred.append(pred_entities.get(offset, "O"))

# Create confusion matrix
labels = sorted(set(y_true + y_pred))
cm = confusion_matrix(y_true, y_pred, labels=labels)

# Plot
df_cm = pd.DataFrame(cm, index=labels, columns=labels)
df_cm






In [ ]:
class_report = classification_report(y_true, y_pred, labels=labels)
print("Classification Report:")
print(class_report)